In [4]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np


In [ ]:
class GAN():
    
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        
        optimizer = Adam(0.0002, 0.5)
        
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        
        self.generator = self.build_generator()
        
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
        
        self.discriminator.trainable = False
        
        validity = self.discriminator(img)
        
        self.combined = Model(z, validity)
        
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)
        
    def build_generator(self):
        
        model = Sequential()
        
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))
        
        model.summary()
        
        noise = Input(shape=(self.latent_dim,))
        img = model(noise)
        
        return Model(noise, img)
    
    
    def build_discriminator(self):
        
        model = Sequential()
        
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()
        
        img = Input(shape=self.img_shape)
        validity = model(img)
        
        return Model(img, validity)
    
    def train(self, epochs, batch_size=128, sample_interval=50):
        
        (X_train, _), (_, _) = mnist.load_data()
        
        X_train = X_train / 127.5 -1
        X_train = np.expand_dims(X_train, axis=3)
        
        valid = np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))
        
        for epoch in range(epochs):
            
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            gen_imgs = self.generator.predict(noise)
            
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            
            g_loss = self.combined.train_on_batch(noise, valid)
            
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
            
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
            
    def sample_images(self, epoch):

        r, c = 5, 5

        noise = np.random.normal(0, 1, (r * c, self.latent_dim))

        gen_imgs = self.generator.predict(noise)

        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0

        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(".\\GAN_mnist\\%d.png" % epoch)
        plt.close()
            
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)
        
        
        

        
        

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_28 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_29 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_40 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

105 [D loss: 0.000525, acc.: 100.00%] [G loss: 0.000824]
106 [D loss: 0.000414, acc.: 100.00%] [G loss: 0.000898]
107 [D loss: 0.000467, acc.: 100.00%] [G loss: 0.000826]
108 [D loss: 0.000408, acc.: 100.00%] [G loss: 0.000729]
109 [D loss: 0.000476, acc.: 100.00%] [G loss: 0.000846]
110 [D loss: 0.000416, acc.: 100.00%] [G loss: 0.000944]
111 [D loss: 0.000401, acc.: 100.00%] [G loss: 0.000843]
112 [D loss: 0.000418, acc.: 100.00%] [G loss: 0.000953]
113 [D loss: 0.000403, acc.: 100.00%] [G loss: 0.000907]
114 [D loss: 0.000359, acc.: 100.00%] [G loss: 0.000825]
115 [D loss: 0.000364, acc.: 100.00%] [G loss: 0.000660]
116 [D loss: 0.000328, acc.: 100.00%] [G loss: 0.000918]
117 [D loss: 0.000374, acc.: 100.00%] [G loss: 0.000689]
118 [D loss: 0.000347, acc.: 100.00%] [G loss: 0.000740]
119 [D loss: 0.000419, acc.: 100.00%] [G loss: 0.000724]
120 [D loss: 0.000349, acc.: 100.00%] [G loss: 0.000704]
121 [D loss: 0.000560, acc.: 100.00%] [G loss: 0.000709]
122 [D loss: 0.000393, acc.: 10

252 [D loss: 0.000112, acc.: 100.00%] [G loss: 0.000215]
253 [D loss: 0.000125, acc.: 100.00%] [G loss: 0.000218]
254 [D loss: 0.000082, acc.: 100.00%] [G loss: 0.000226]
255 [D loss: 0.000092, acc.: 100.00%] [G loss: 0.000178]
256 [D loss: 0.000107, acc.: 100.00%] [G loss: 0.000206]
257 [D loss: 0.000070, acc.: 100.00%] [G loss: 0.000230]
258 [D loss: 0.000108, acc.: 100.00%] [G loss: 0.000174]
259 [D loss: 0.000092, acc.: 100.00%] [G loss: 0.000220]
260 [D loss: 0.000090, acc.: 100.00%] [G loss: 0.000147]
261 [D loss: 0.000102, acc.: 100.00%] [G loss: 0.000181]
262 [D loss: 0.000129, acc.: 100.00%] [G loss: 0.000187]
263 [D loss: 0.000082, acc.: 100.00%] [G loss: 0.000203]
264 [D loss: 0.000087, acc.: 100.00%] [G loss: 0.000175]
265 [D loss: 0.000101, acc.: 100.00%] [G loss: 0.000173]
266 [D loss: 0.000101, acc.: 100.00%] [G loss: 0.000124]
267 [D loss: 0.000110, acc.: 100.00%] [G loss: 0.000232]
268 [D loss: 0.000111, acc.: 100.00%] [G loss: 0.000174]
269 [D loss: 0.000097, acc.: 10

397 [D loss: 0.000048, acc.: 100.00%] [G loss: 0.000108]
398 [D loss: 0.000046, acc.: 100.00%] [G loss: 0.000093]
399 [D loss: 0.000051, acc.: 100.00%] [G loss: 0.000102]
400 [D loss: 0.000051, acc.: 100.00%] [G loss: 0.000099]
401 [D loss: 0.000061, acc.: 100.00%] [G loss: 0.000073]
402 [D loss: 0.000043, acc.: 100.00%] [G loss: 0.000079]
403 [D loss: 0.000057, acc.: 100.00%] [G loss: 0.000081]
404 [D loss: 0.000041, acc.: 100.00%] [G loss: 0.000086]
405 [D loss: 0.000053, acc.: 100.00%] [G loss: 0.000064]
406 [D loss: 0.000046, acc.: 100.00%] [G loss: 0.000067]
407 [D loss: 0.000047, acc.: 100.00%] [G loss: 0.000100]
408 [D loss: 0.000047, acc.: 100.00%] [G loss: 0.000088]
409 [D loss: 0.000043, acc.: 100.00%] [G loss: 0.000092]
410 [D loss: 0.000035, acc.: 100.00%] [G loss: 0.000075]
411 [D loss: 0.000064, acc.: 100.00%] [G loss: 0.000095]
412 [D loss: 0.000037, acc.: 100.00%] [G loss: 0.000081]
413 [D loss: 0.000042, acc.: 100.00%] [G loss: 0.000083]
414 [D loss: 0.000057, acc.: 10

546 [D loss: 0.000025, acc.: 100.00%] [G loss: 0.000062]
547 [D loss: 0.000023, acc.: 100.00%] [G loss: 0.000048]
548 [D loss: 0.000026, acc.: 100.00%] [G loss: 0.000049]
549 [D loss: 0.000031, acc.: 100.00%] [G loss: 0.000050]
550 [D loss: 0.000030, acc.: 100.00%] [G loss: 0.000057]
551 [D loss: 0.000028, acc.: 100.00%] [G loss: 0.000044]
552 [D loss: 0.000023, acc.: 100.00%] [G loss: 0.000061]
553 [D loss: 0.000029, acc.: 100.00%] [G loss: 0.000046]
554 [D loss: 0.000034, acc.: 100.00%] [G loss: 0.000064]
555 [D loss: 0.000019, acc.: 100.00%] [G loss: 0.000045]
556 [D loss: 0.000025, acc.: 100.00%] [G loss: 0.000049]
557 [D loss: 0.000027, acc.: 100.00%] [G loss: 0.000061]
558 [D loss: 0.000036, acc.: 100.00%] [G loss: 0.000050]
559 [D loss: 0.000038, acc.: 100.00%] [G loss: 0.000058]
560 [D loss: 0.000020, acc.: 100.00%] [G loss: 0.000057]
561 [D loss: 0.000023, acc.: 100.00%] [G loss: 0.000044]
562 [D loss: 0.000034, acc.: 100.00%] [G loss: 0.000046]
563 [D loss: 0.000027, acc.: 10

...

18852 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18853 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18854 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18855 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18856 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18857 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18858 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18859 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18860 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18861 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18862 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18863 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18864 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18865 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18866 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18867 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
18868 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.00000

19000 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19001 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19002 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19003 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19004 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19005 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19006 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19007 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19008 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19009 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19010 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19011 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19012 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19013 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19014 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19015 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19016 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.00000

19143 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19144 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19145 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19146 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19147 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19148 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19149 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19150 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19151 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19152 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19153 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19154 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19155 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19156 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19157 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19158 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19159 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.00000

19290 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19291 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19292 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19293 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19294 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19295 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19296 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19297 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19298 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19299 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19300 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19301 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19302 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19303 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19304 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19305 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19306 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.00000

19438 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19439 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19440 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19441 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19442 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19443 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19444 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19445 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19446 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19447 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19448 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19449 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19450 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19451 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19452 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19453 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.000000]
19454 [D loss: 0.000000, acc.: 100.00%] [G loss: 0.00000